## 0. Get the code map

In [3]:
import pandas as pd
import pickle
import numpy as np
import json

In [4]:
icd9_data = pd.read_csv("/home/leyang.sun/BERHT/phecode1.csv")
icd10_data = pd.read_csv("/home/leyang.sun/BERHT/phecode2.csv")
icd9_df = icd9_data[["phecode", "phenotype"]]
new_Col = ["ICD9", "Phenotypes" ]
icd9_df.columns  = new_Col
print(icd9_df.head(20))
print(icd9_df["ICD9"].dtype) # need to convert to string

     ICD9                                   Phenotypes
0    8.00                         Intestinal infection
1    8.50                          Bacterial enteritis
2    8.51                            Intestinal e.coli
3    8.52     Intestinal infection due to C. difficile
4    8.60                              Viral Enteritis
5    8.70         Intestinal infection due to protozoa
6   10.00                                 Tuberculosis
7   31.00           Diseases due to other mycobacteria
8   31.10                                      Leprosy
9   38.00                                   Septicemia
10  38.10                     Gram negative septicemia
11  38.20                     Gram positive septicemia
12  38.30                                   Bacteremia
13  41.00                      Bacterial infection NOS
14  41.10                    Staphylococcus infections
15  41.11  Methicillin sensitive Staphylococcus aureus
16  41.12  Methicillin resistant Staphylococcus aureus
17  41.20 

In [5]:
icd10_df = icd10_data[["PHECODE", 'ICD10', 'Excl. Phenotypes']]
new_Col = ["ICD9", "ICD10", "Phenotypes" ]
icd10_df.columns  = new_Col
print(icd10_df.head(20))
print(icd10_df["ICD9"].dtype) # object, need to convert both to string

    ICD9  ICD10            Phenotypes
0    8.0    A00  Intestinal infection
1    8.0  A00.0  Intestinal infection
2    8.0  A00.1  Intestinal infection
3    8.0  A00.9  Intestinal infection
4    8.0    A01  Intestinal infection
5    8.5  A01.0  Intestinal infection
6    8.0  A01.1  Intestinal infection
7    8.0  A01.2  Intestinal infection
8    8.0  A01.3  Intestinal infection
9    8.0  A01.4  Intestinal infection
10   8.5    A02  Intestinal infection
11   8.5  A02.0  Intestinal infection
12  38.1  A02.1   bacterial infection
13   8.5  A02.2  Intestinal infection
14   8.5  A02.8  Intestinal infection
15   8.5  A02.9  Intestinal infection
16   8.5    A03  Intestinal infection
17   8.5  A03.0  Intestinal infection
18   8.5  A03.1  Intestinal infection
19   8.5  A03.2  Intestinal infection
float64


In [6]:
df_merged = pd.merge(icd9_df, icd10_df, on='Phenotypes',how='outer')
df_merged = df_merged.astype({'ICD9_x':'string', 'ICD9_y':'string', 'ICD10':'string'})
df_merged = df_merged.fillna("nan")

In [7]:
print(df_merged.head(5))

  ICD9_x            Phenotypes ICD9_y  ICD10
0    8.0  Intestinal infection    8.0    A00
1    8.0  Intestinal infection    8.0  A00.0
2    8.0  Intestinal infection    8.0  A00.1
3    8.0  Intestinal infection    8.0  A00.9
4    8.0  Intestinal infection    8.0    A01


In [8]:
# Create a mapping dictionary
mapping_dict = {}
mapped_number = 0
for index, row in df_merged.iterrows():
    
    disease = row['Phenotypes']
    icd9_1 = row['ICD9_x']
    icd9_2 = row['ICD9_y']
    icd10 = row['ICD10']

   
    if disease in mapping_dict.keys():
        disease_code = mapping_dict[disease]
        
        if icd9_1 not in disease_code['ICD9_1']:
            disease_code['ICD9_1'].append(icd9_1)
        
        if icd9_2 not in disease_code['ICD9_2']:
            disease_code['ICD9_2'].append(icd9_2)
            
        if icd10 not in disease_code['ICD10']:
            disease_code['ICD10'].append(icd10)
       
   
    else:
        mapped_number += 1
        mapping_dict[disease] = {
            'ICD9_1': [icd9_1],
            'ICD9_2': [icd9_2],
            'ICD10': [icd10],
            'MappedNumber': mapped_number
        }


In [9]:

# Example usage: Given an ICD9 or ICD10 code, identify the mapped number
code_to_find = '41.12'  # Replace with the code you want to search for, should be string

mapped_number = None
for disease, mapping in mapping_dict.items():
    if code_to_find in mapping['ICD9_1']  or code_to_find in mapping['ICD9_2'] or code_to_find in mapping['ICD10'] :
        mapped_number = mapping['MappedNumber']
        break
    

if mapped_number is not None:
    print(f"The mapped number for the code '{code_to_find}' is {mapped_number}")
else:
    print(f"Code '{code_to_find}' not found in the mapping.")

The mapped number for the code '41.12' is 17


In [10]:
# save the dictionary into binary mode
file_path = 'mapping_dict.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(mapping_dict, file)
file.close()

In [11]:
# Open the file in binary read mode
with open(file_path, 'rb') as file:
    mapping_dict = pickle.load(file)
# Close the file
file.close()

## 1. Extract the birth date of the patients from the patient_dimension.cv

In [12]:
import pandas as pd
import pickle

df_patient_dimension = pd.read_csv("/data/datasets/Tianchen/data_from_old_server/2021/ADRD_data_from_Xi/i2b2/patient_dimension.csv")

In [13]:
print(df_patient_dimension.head)

<bound method NDFrame.head of                   deid_pat_ID VITAL_STATUS_CD  BIRTH_DATE  DEATH_DATE  SEX_CD  \
0        IRB202001139_PAT_989               N  1931-11-13         NaN    Male   
1        IRB202001139_PAT_730               N  1924-06-24         NaN  Female   
2        IRB202001139_PAT_269               Y  1933-04-20  2018-12-03  Female   
3        IRB202001139_PAT_737               N  1971-02-10         NaN  Female   
4        IRB202001139_PAT_679               N  1964-12-27         NaN  Female   
...                       ...             ...         ...         ...     ...   
52189  IRB202001139_PAT_49128               N  1980-03-02         NaN  Female   
52190  IRB202001139_PAT_49106               N  2003-01-16         NaN    Male   
52191  IRB202001139_PAT_49180               N  1931-03-06         NaN  Female   
52192  IRB202001139_PAT_49240               N  2000-09-25         NaN  Female   
52193  IRB202001139_PAT_49255               N  1968-01-26         NaN  Female  

## 2. Calculate the age of the patients at each encounter with the START_DATE under DIAGNOSIS.csv

In [15]:
DIAGNOSES = pd.read_csv("/data/datasets/Tianchen/data_from_old_server/2021/ADRD_data_from_Xi/process_observation/process_observation/DIAGNOSES.csv")

/tmp/ipykernel_39858/1002630756.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  DIAGNOSES = pd.read_csv("/data/datasets/Tianchen/data_from_old_server/2021/ADRD_data_from_Xi/process_observation/process_observation/DIAGNOSES.csv")


In [16]:
print(DIAGNOSES.head)

<bound method NDFrame.head of                      deid_pat_ID               deid_enc_ID     CONCEPT_CD  \
0           IRB202001139_PAT_626    IRB202001139_ENC_42808  COMOR:ALCOHOL   
1           IRB202001139_PAT_862    IRB202001139_ENC_33331  COMOR:ALCOHOL   
2            IRB202001139_PAT_27    IRB202001139_ENC_40434  COMOR:ALCOHOL   
3           IRB202001139_PAT_165    IRB202001139_ENC_33614  COMOR:ALCOHOL   
4           IRB202001139_PAT_333     IRB202001139_ENC_3261  COMOR:ALCOHOL   
...                          ...                       ...            ...   
44714088  IRB202001139_PAT_49258  IRB202001139_ENC_3630261  ICD9-D:366.10   
44714089  IRB202001139_PAT_49258  IRB202001139_ENC_3630333  ICD9-D:338.29   
44714090  IRB202001139_PAT_49258  IRB202001139_ENC_3630284  ICD9-D:278.01   
44714091  IRB202001139_PAT_49258  IRB202001139_ENC_3630154  ICD9-D:278.00   
44714092  IRB202001139_PAT_49258  IRB202001139_ENC_3630456  ICD9-D:V67.00   

          START_DATE         MODIFIER_CD  VAL

## Filter pid missing in either one of the df

In [17]:

# Step 1: Merge the dataframes to find common patients
merged_df = df_patient_dimension.merge(DIAGNOSES, on='deid_pat_ID', how='inner')


In [18]:
print(merged_df.head(5))

            deid_pat_ID VITAL_STATUS_CD  BIRTH_DATE DEATH_DATE  SEX_CD  \
0  IRB202001139_PAT_989               N  1931-11-13        NaN    Male   
1  IRB202001139_PAT_989               N  1931-11-13        NaN    Male   
2  IRB202001139_PAT_730               N  1924-06-24        NaN  Female   
3  IRB202001139_PAT_730               N  1924-06-24        NaN  Female   
4  IRB202001139_PAT_730               N  1924-06-24        NaN  Female   

   AGE_IN_YEARS_NUM LANGUAGE_CD RACE_CD MARITAL_STATUS_CD  RELIGION_CD  ...  \
0                88     English   White            Single          NaN  ...   
1                88     English   White            Single          NaN  ...   
2                95     English   Other           Married          NaN  ...   
3                95     English   Other           Married          NaN  ...   
4                95     English   Other           Married          NaN  ...   

  NVAL_NUM  VALUEFLAG_CD  QUANTITY_NUM UNITS_CD    END_DATE  LOCATION_CD  \
0   

In [19]:

# Step 3: Merge dataframes on 'pid' to get birth_date for each encounter
#merged_df = pd.merge(DIAGNOSES, df_patient_dimension[['deid_pat_ID', 'BIRTH_DATE']], on='deid_pat_ID')
merged_df['START_DATE'] = pd.to_datetime(merged_df['START_DATE'])
merged_df['BIRTH_DATE'] = pd.to_datetime(merged_df['BIRTH_DATE'])
merged_df['age_at_encounter'] = (merged_df['START_DATE'] - merged_df['BIRTH_DATE']).dt.days // 365


In [20]:
print(merged_df.index)

RangeIndex(start=0, stop=54887632, step=1)


In [21]:
selected_columns = ['deid_pat_ID', 'deid_enc_ID',"CONCEPT_CD",'age_at_encounter']

# Create a new DataFrame with the selected columns
merge_df_selected = merged_df[selected_columns]
merge_df_selected = merge_df_selected[~merge_df_selected['CONCEPT_CD'].str.startswith('COMOR')]

# Assuming you have a DataFrame df with a column named 'ICD Code'
# Replace 'ICD Code' with the actual column name if it's different in your DataFrame
merge_df_selected['CONCEPT_CD'] = merge_df_selected['CONCEPT_CD'].str.split(':').str[1]

# This code splits the string by ':' and selects the second part (index 1), which is the ICD code.

# If you want to remove leading or trailing spaces, you can also strip the resulting values:
merge_df_selected['CONCEPT_CD'] = merge_df_selected['CONCEPT_CD'].str.strip()

In [22]:
merge_df_selected = merge_df_selected.drop_duplicates()
print(merge_df_selected.index)
print(merge_df_selected.head(10))

Index([       0,        2,        4,        8,       12,       17,       19,
             20,       21,       22,
       ...
       54887609, 54887610, 54887611, 54887612, 54887618, 54887619, 54887620,
       54887624, 54887625, 54887630],
      dtype='int64', length=13561866)
             deid_pat_ID               deid_enc_ID CONCEPT_CD  \
0   IRB202001139_PAT_989    IRB202001139_ENC_38294     331.83   
2   IRB202001139_PAT_730    IRB202001139_ENC_38047     294.20   
4   IRB202001139_PAT_730    IRB202001139_ENC_40859     294.20   
8   IRB202001139_PAT_730    IRB202001139_ENC_38047      298.8   
12  IRB202001139_PAT_730    IRB202001139_ENC_38047      458.9   
17  IRB202001139_PAT_730    IRB202001139_ENC_19794      692.9   
19  IRB202001139_PAT_730  IRB202001139_ENC_1924698      692.9   
20  IRB202001139_PAT_730    IRB202001139_ENC_19796      692.9   
21  IRB202001139_PAT_730    IRB202001139_ENC_40859      692.9   
22  IRB202001139_PAT_730    IRB202001139_ENC_15150      692.9   

    ag

In [23]:
unique_patients_count = merge_df_selected['deid_pat_ID'].nunique()
print("Number of unique deid_pat_ID:", unique_patients_count)

Number of unique deid_pat_ID: 48903


In [ ]:
# add the mapping code column
merge_df_selected['map_code_v3'] = merge_df_selected['CONCEPT_CD'].apply(find_mapped_number_v3)
# extract the rows without mapped code
df_no_map_code_all = merge_df_selected[merge_df_selected['map_code_v3'].isna()]


In [ ]:
df_no_map_code_all.to_csv("/data/datasets/leyang.sun/df_no_map_code_all.csv")

## Randomly sample 10,000 patients record

In [24]:
unique_patients = merge_df_selected['deid_pat_ID'].unique()
np.random.seed(42)
np.random.shuffle(unique_patients)

# Sample 10,000 unique patients
sampled_patients = unique_patients[:10000]

# Create a new DataFrame containing only the sampled patients
sampled_df = merge_df_selected[merge_df_selected['deid_pat_ID'].isin(sampled_patients)]

In [25]:
print(sampled_df.head)

<bound method NDFrame.head of                      deid_pat_ID               deid_enc_ID CONCEPT_CD  \
2           IRB202001139_PAT_730    IRB202001139_ENC_38047     294.20   
4           IRB202001139_PAT_730    IRB202001139_ENC_40859     294.20   
8           IRB202001139_PAT_730    IRB202001139_ENC_38047      298.8   
12          IRB202001139_PAT_730    IRB202001139_ENC_38047      458.9   
17          IRB202001139_PAT_730    IRB202001139_ENC_19794      692.9   
...                          ...                       ...        ...   
54887079  IRB202001139_PAT_49106  IRB202001139_ENC_3630503   V86.09XA   
54887081  IRB202001139_PAT_49106  IRB202001139_ENC_3630503   V86.59XA   
54887086  IRB202001139_PAT_49106  IRB202001139_ENC_3630503   V86.99XA   
54887088  IRB202001139_PAT_49106  IRB202001139_ENC_3630503    Z00.129   
54887094  IRB202001139_PAT_49106  IRB202001139_ENC_4399182     Z46.82   

          age_at_encounter  
2                       89  
4                       87  
8     

In [27]:
def revise_code_to_two_digit(code): # the input code has 2 digits
    parts = code.split('.')
    if len(parts) > 1:
        revised_code = parts[0] + '.' + parts[1][:2]  # Keep the first two digits after the dot
        return revised_code
    return code

def revise_code_to_one_digit(code): # the input code has 2 digits
    parts = code.split('.')
    if len(parts) > 1:
        revised_code = parts[0] + '.' + parts[1][:1]  # Keep the first digit after the dot
        return revised_code
    return code

def revise_code_to_integer(code): # the input code has 1 digit
    parts = code.split('.')
    if len(parts) > 1:
        revised_code = parts[0] # keep the integer part
        return revised_code
    return code
    

In [27]:
print(revise_code_to_two_digit("Z87.891"))

Z87.89


In [28]:
# exact mapping
# define the mapping function to perform on a whole dataframe
def find_mapped_number_v1(code_to_find):
    # try to find the mapped code without revising the code
    for disease, mapping in mapping_dict.items():
        if code_to_find in mapping['ICD9_1'] or code_to_find in mapping['ICD9_2'] or code_to_find in mapping['ICD10']:
            return mapping['MappedNumber']
                
    return None

In [29]:
# # define the mapping function to perform on a whole dataframe
# def find_mapped_number_v3(code_to_find):
#     # try to find the mapped code without revising the code
#     for disease, mapping in mapping_dict.items():
#         if code_to_find in mapping['ICD9_1'] or code_to_find in mapping['ICD9_2'] or code_to_find in mapping['ICD10']:
#             return mapping['MappedNumber']
    
#     # try to revise the code and find the map again
#     parts = code_to_find.split('.')
#     if len(parts)> 1:
#         revised_code = revise_code_to_integer(code_to_find)
                        
#         for disease, mapping in mapping_dict.items():
#             if revised_code in mapping['ICD9_1'] or revised_code in mapping['ICD9_2'] or revised_code in mapping['ICD10']:
#                 return mapping['MappedNumber']
                
#     return None

In [25]:
def find_mapped_number_v3(code_to_find):
    # Check if the exact code_to_find exists in the mapping
    for disease, mapping in mapping_dict.items():
        for icd_code in [mapping['ICD9_1'], mapping['ICD9_2'], mapping['ICD10']]:
            for mapped_code in icd_code:
                if code_to_find == mapped_code:
                    #print(mapped_code)
                    return mapping['MappedNumber']
               
    # If the exact code is not found, try variations 1: code_to_find: 041.12 -> 41.12; 010 -> 10
    revised_code_to_find = code_to_find.lstrip('0')
    for disease, mapping in mapping_dict.items():
        for icd_code in [mapping['ICD9_1'], mapping['ICD9_2'], mapping['ICD10']]:
            for mapped_code in icd_code:
                if revised_code_to_find == mapped_code:
                    #print(mapped_code)
                    return mapping['MappedNumber']
    
    # If the exact code is not found, try variations 2: code_to_find: 10 -> 10.0
    revised_code_to_find = revised_code_to_find  + '.0'
    for disease, mapping in mapping_dict.items():
        for icd_code in [mapping['ICD9_1'], mapping['ICD9_2'], mapping['ICD10']]:
            for mapped_code in icd_code:
                if revised_code_to_find == mapped_code:
                    #print(mapped_code)
                    return mapping['MappedNumber']    
                
    # if both the code variations not found, merge to the major branch 
    parts = code_to_find.split('.')
    if len(parts)> 1:
        branch_code = revise_code_to_integer(code_to_find)
                        
        # retry exact finding
        for disease, mapping in mapping_dict.items():
            for icd_code in [mapping['ICD9_1'], mapping['ICD9_2'], mapping['ICD10']]:
                for mapped_code in icd_code:
                    if branch_code == mapped_code:
                        #print(mapped_code)
                        return mapping['MappedNumber']
            
            
        # retry variations for branch code     
        revised_branch_code = branch_code.lstrip('0')
        for disease, mapping in mapping_dict.items():
            for icd_code in [mapping['ICD9_1'], mapping['ICD9_2'], mapping['ICD10']]:
                for mapped_code in icd_code:
                    if revised_branch_code == mapped_code:
                        #print(mapped_code)
                        return mapping['MappedNumber']

        # If the exact code is not found, try variations 2: code_to_find: 10 -> 10.0
        revised_branch_code = revised_branch_code  + '.0'
        for disease, mapping in mapping_dict.items():
            for icd_code in [mapping['ICD9_1'], mapping['ICD9_2'], mapping['ICD10']]:
                for mapped_code in icd_code:
                    if revised_branch_code == mapped_code:
                        #print(mapped_code)
                        return mapping['MappedNumber']        
           
                
    return None

    # Check if the exact code_to_find exists in the mapping
    for disease, mapping in mapping_dict.items():
        for icd_code in [mapping['ICD9_1'], mapping['ICD9_2'], mapping['ICD10']]:
            for mapped_code in icd_code:
                if code_to_find == mapped_code:
                    #print(mapped_code)
                    return mapping['MappedNumber']
               
    # If the exact code is not found, try variations 1: code_to_find: 041.12 -> 41.12; 010 -> 10
    revised_code_to_find = code_to_find.lstrip('0')
    for disease, mapping in mapping_dict.items():
        for icd_code in [mapping['ICD9_1'], mapping['ICD9_2'], mapping['ICD10']]:
            for mapped_code in icd_code:
                if revised_code_to_find == mapped_code:
                    #print(mapped_code)
                    return mapping['MappedNumber']
    
    # If the exact code is not found, try variations 2: code_to_find: 10 -> 10.0
    revised_code_to_find = revised_code_to_find  + '.0'
    for disease, mapping in mapping_dict.items():
        for icd_code in [mapping['ICD9_1'], mapping['ICD9_2'], mapping['ICD10']]:
            for mapped_code in icd_code:
                if revised_code_to_find == mapped_code:
                   # print(mapped_code)
                    return mapping['MappedNumber']    
                
    # if both the code variations not found, merge to the major branch 
    parts = code_to_find.split('.')
    if len(parts)> 1:
        branch_code = revise_code_to_integer(code_to_find)
                        
        # retry exact finding
        for disease, mapping in mapping_dict.items():
            for icd_code in [mapping['ICD9_1'], mapping['ICD9_2'], mapping['ICD10']]:
                for mapped_code in icd_code:
                    if branch_code == mapped_code:
                       # print(mapped_code)
                        return mapping['MappedNumber']
            
            
        # retry variations for branch code     
        revised_branch_code = branch_code.lstrip('0')
        for disease, mapping in mapping_dict.items():
            for icd_code in [mapping['ICD9_1'], mapping['ICD9_2'], mapping['ICD10']]:
                for mapped_code in icd_code:
                    if revised_branch_code == mapped_code:
                       # print(mapped_code)
                        return mapping['MappedNumber']

        # If the exact code is not found, try variations 2: code_to_find: 10 -> 10.0
        revised_branch_code = revised_branch_code  + '.0'
        for disease, mapping in mapping_dict.items():
            for icd_code in [mapping['ICD9_1'], mapping['ICD9_2'], mapping['ICD10']]:
                for mapped_code in icd_code:
                    if revised_branch_code == mapped_code:
                        #print(mapped_code)
                        return mapping['MappedNumber']        
           
                
    return None


In [37]:
# code = 'R11.20'
# parts = code.split('.')
# print(len(parts))
# print(len(parts[0]))

In [38]:
# # define the mapping function to perform on a whole dataframe
# def find_mapped_number_v2(code_to_find):
#     # try to find the mapped code without revising the code
#     for disease, mapping in mapping_dict.items():
#         if code_to_find in mapping['ICD9_1'] or code_to_find in mapping['ICD9_2'] or code_to_find in mapping['ICD10']:
#             return mapping['MappedNumber']
    
#     # try to revise the code and find the map again
#     parts = code_to_find.split('.')
#     if len(parts) > 1:
#         if len(parts[1]) > 1: # in the format of "'R11.20'"
#             revised_code = revise_code_to_one_digit(code_to_find)
#             for disease, mapping in mapping_dict.items():
#                 if revised_code in mapping['ICD9_1'] or revised_code in mapping['ICD9_2'] or revised_code in mapping['ICD10']:
#                     return mapping['MappedNumber']
#                 else:
#                     revised_code = revise_code_to_integer(revised_code) # in the format of "'R11.2'"
#                     for disease, mapping in mapping_dict.items():
#                         if revised_code in mapping['ICD9_1'] or revised_code in mapping['ICD9_2'] or revised_code in mapping['ICD10']:
#                             return mapping['MappedNumber']
                        
#         elif len(parts[1]) == 1: # in the format of "'R11.2'"
#             revised_code = revise_code_to_integer(code_to_find)
#             for disease, mapping in mapping_dict.items():
#                 if revised_code in mapping['ICD9_1'] or revised_code in mapping['ICD9_2'] or revised_code in mapping['ICD10']:
#                     return mapping['MappedNumber']
                
#     return None

In [39]:
# test_df = sampled_df.head(10)

In [42]:
# add the mapping code column
sampled_df['map_code_v3'] = sampled_df['CONCEPT_CD'].apply(find_mapped_number_v3)

/tmp/ipykernel_95315/1555334246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df['map_code_v3'] = sampled_df['CONCEPT_CD'].apply(find_mapped_number_v3)


In [43]:
print(sampled_df.head)

<bound method NDFrame.head of                      deid_pat_ID               deid_enc_ID CONCEPT_CD  \
2           IRB202001139_PAT_730    IRB202001139_ENC_38047     294.20   
4           IRB202001139_PAT_730    IRB202001139_ENC_40859     294.20   
8           IRB202001139_PAT_730    IRB202001139_ENC_38047      298.8   
12          IRB202001139_PAT_730    IRB202001139_ENC_38047      458.9   
17          IRB202001139_PAT_730    IRB202001139_ENC_19794      692.9   
...                          ...                       ...        ...   
54887079  IRB202001139_PAT_49106  IRB202001139_ENC_3630503   V86.09XA   
54887081  IRB202001139_PAT_49106  IRB202001139_ENC_3630503   V86.59XA   
54887086  IRB202001139_PAT_49106  IRB202001139_ENC_3630503   V86.99XA   
54887088  IRB202001139_PAT_49106  IRB202001139_ENC_3630503    Z00.129   
54887094  IRB202001139_PAT_49106  IRB202001139_ENC_4399182     Z46.82   

          age_at_encounter  map_code_v3  
2                       89          NaN  
4        

In [44]:
# extract the rows without mapped code
df_no_map_code = sampled_df[sampled_df['map_code_v3'].isna()]


In [45]:
print(df_no_map_code.index)

Index([       2,        4,        8,       17,       19,       20,       21,
             22,     1328,     1330,
       ...
       54883942, 54883949, 54883951, 54883952, 54883955, 54887079, 54887081,
       54887086, 54887088, 54887094],
      dtype='int64', length=647738)


In [46]:
df_no_map_code.to_csv("/data/datasets/leyang.sun/df_no_map_code.csv")

In [47]:
print(df_no_map_code.head)

<bound method NDFrame.head of                      deid_pat_ID               deid_enc_ID CONCEPT_CD  \
2           IRB202001139_PAT_730    IRB202001139_ENC_38047     294.20   
4           IRB202001139_PAT_730    IRB202001139_ENC_40859     294.20   
8           IRB202001139_PAT_730    IRB202001139_ENC_38047      298.8   
17          IRB202001139_PAT_730    IRB202001139_ENC_19794      692.9   
19          IRB202001139_PAT_730  IRB202001139_ENC_1924698      692.9   
...                          ...                       ...        ...   
54887079  IRB202001139_PAT_49106  IRB202001139_ENC_3630503   V86.09XA   
54887081  IRB202001139_PAT_49106  IRB202001139_ENC_3630503   V86.59XA   
54887086  IRB202001139_PAT_49106  IRB202001139_ENC_3630503   V86.99XA   
54887088  IRB202001139_PAT_49106  IRB202001139_ENC_3630503    Z00.129   
54887094  IRB202001139_PAT_49106  IRB202001139_ENC_4399182     Z46.82   

          age_at_encounter  map_code_v3  
2                       89          NaN  
4        

In [48]:
# drop rows with NaN in codemap
merge_df_selected_dropna = sampled_df.dropna(subset=['map_code_v3'])

In [49]:
print(merge_df_selected_dropna.index)
print(merge_df_selected_dropna.head(10))

Index([      12,       28,       29,       32,       34,       37,       42,
             44,      760,      761,
       ...
       54887044, 54887046, 54887049, 54887051, 54887054, 54887056, 54887065,
       54887073, 54887075, 54887077],
      dtype='int64', length=2144408)
              deid_pat_ID             deid_enc_ID CONCEPT_CD  \
12   IRB202001139_PAT_730  IRB202001139_ENC_38047      458.9   
28   IRB202001139_PAT_730  IRB202001139_ENC_40859     695.89   
29   IRB202001139_PAT_730  IRB202001139_ENC_19796     695.89   
32   IRB202001139_PAT_730  IRB202001139_ENC_15150      695.9   
34   IRB202001139_PAT_730  IRB202001139_ENC_15150      709.9   
37   IRB202001139_PAT_730  IRB202001139_ENC_38047     780.79   
42   IRB202001139_PAT_730  IRB202001139_ENC_25578      782.1   
44   IRB202001139_PAT_730  IRB202001139_ENC_15150      782.1   
760  IRB202001139_PAT_679   IRB202001139_ENC_8973      E04.9   
761  IRB202001139_PAT_679  IRB202001139_ENC_40194      E04.9   

     age_at_encoun

In [50]:

# Sort within each patient group by 'age_at_encounter'
merge_df_selected_dropna_sorted = merge_df_selected_dropna.sort_values(by=['deid_pat_ID', 'age_at_encounter'])

# Display the sorted DataFrame
# print(merge_df_selected_sorted.head(50))

In [51]:
print(merge_df_selected_dropna_sorted.index)

Index([  107652,   107658,   107671,   107675,   107688,   107689,   107692,
         107695,   107700,   107716,
       ...
       47278244, 47278246, 47278247, 47278252, 47278253, 47278254, 47278255,
       47278256, 47278257, 47278267],
      dtype='int64', length=2144408)


In [52]:

# Remove duplicate encounters to prepare for age column for each patient, the df for age extraction is merge_df_age
merge_df_age = merge_df_selected_dropna_sorted.drop_duplicates(subset=['deid_pat_ID', 'deid_enc_ID'])
print(merge_df_age.index)

Index([  107652,   107658,   107688,   107689,   107718,   107731,   107425,
         107410,   107448,   107450,
       ...
       47278186, 47278194, 47278211, 47278246, 47278247, 47278252, 47278253,
       47278255, 47278256, 47278257],
      dtype='int64', length=493693)


In [53]:
# get age_vector for each patient

# Group by 'pid' and aggregate 'age_at_encounter' into a list
patient_age_df = merge_df_age.groupby('deid_pat_ID')['age_at_encounter'].agg(list).reset_index()

# Rename the column to 'age_vector'
patient_age_df = patient_age_df.rename(columns={'age_at_encounter': 'age_vector'})

# Display the resulting dataframe
print(patient_age_df.head)

<bound method NDFrame.head of                  deid_pat_ID  \
0         IRB202001139_PAT_1   
1        IRB202001139_PAT_10   
2     IRB202001139_PAT_10001   
3     IRB202001139_PAT_10002   
4     IRB202001139_PAT_10009   
...                      ...   
9916   IRB202001139_PAT_9958   
9917   IRB202001139_PAT_9965   
9918   IRB202001139_PAT_9977   
9919   IRB202001139_PAT_9992   
9920   IRB202001139_PAT_9993   

                                             age_vector  
0     [31, 31, 31, 31, 31, 31, 32, 33, 33, 33, 33, 3...  
1     [69, 69, 70, 70, 70, 70, 70, 70, 71, 71, 71, 7...  
2     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
3     [19, 19, 20, 20, 21, 21, 21, 22, 22, 22, 22, 2...  
4     [66, 66, 66, 66, 67, 67, 67, 67, 67, 67, 67, 6...  
...                                                 ...  
9916                               [62, 62, 62, 62, 62]  
9917  [77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 77, 7...  
9918  [9, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12...  
9919 

## 3. Aggregate the diagnosis codes of the patients under the same encounter defined by deid_enc_ID under DIAGNOSIS.csv. 

In [54]:
print(merge_df_selected_dropna_sorted.head)

<bound method NDFrame.head of                     deid_pat_ID               deid_enc_ID CONCEPT_CD  \
107652       IRB202001139_PAT_1    IRB202001139_ENC_31294      276.2   
107658       IRB202001139_PAT_1  IRB202001139_ENC_1925177      276.2   
107671       IRB202001139_PAT_1    IRB202001139_ENC_31294     276.69   
107675       IRB202001139_PAT_1    IRB202001139_ENC_31294      285.9   
107688       IRB202001139_PAT_1  IRB202001139_ENC_1923892      344.9   
...                         ...                       ...        ...   
47278254  IRB202001139_PAT_9993  IRB202001139_ENC_3795741     793.19   
47278255  IRB202001139_PAT_9993  IRB202001139_ENC_3795722     793.19   
47278256  IRB202001139_PAT_9993  IRB202001139_ENC_3795724     793.19   
47278257  IRB202001139_PAT_9993  IRB202001139_ENC_3795687     793.19   
47278267  IRB202001139_PAT_9993  IRB202001139_ENC_2914116     794.31   

          age_at_encounter  map_code_v3  
107652                  31        366.0  
107658               

In [55]:
diagnosis_selected_columns = ['deid_pat_ID', 'deid_enc_ID',"map_code_v3"]


# Create a new DataFrame with the selected columns
merge_df_diagnosis = merge_df_selected_dropna_sorted[diagnosis_selected_columns]

In [56]:
merge_df_diagnosis = merge_df_diagnosis.reset_index(drop=True)
print(merge_df_diagnosis.head)

<bound method NDFrame.head of                    deid_pat_ID               deid_enc_ID  map_code_v3
0           IRB202001139_PAT_1    IRB202001139_ENC_31294        366.0
1           IRB202001139_PAT_1  IRB202001139_ENC_1925177        366.0
2           IRB202001139_PAT_1    IRB202001139_ENC_31294        366.0
3           IRB202001139_PAT_1    IRB202001139_ENC_31294        418.0
4           IRB202001139_PAT_1  IRB202001139_ENC_1923892        579.0
...                        ...                       ...          ...
2144403  IRB202001139_PAT_9993  IRB202001139_ENC_3795741       1717.0
2144404  IRB202001139_PAT_9993  IRB202001139_ENC_3795722       1717.0
2144405  IRB202001139_PAT_9993  IRB202001139_ENC_3795724       1717.0
2144406  IRB202001139_PAT_9993  IRB202001139_ENC_3795687       1717.0
2144407  IRB202001139_PAT_9993  IRB202001139_ENC_2914116       1719.0

[2144408 rows x 3 columns]>


In [57]:
# Initialize an empty dictionary to store aggregated vectors
pid_diagnosis_dict = {}

# Iterate through each row of the DataFrame
for index, row in merge_df_diagnosis.iterrows():
    pid = row['deid_pat_ID']
    encounter_id = row['deid_enc_ID']
    code = row['map_code_v3']

    # Check if the pid is already in the dictionary
    if pid in pid_diagnosis_dict:
        # Check if encounter_id is already in the dictionary
        if encounter_id in pid_diagnosis_dict[pid]:
            # Append the diagnosis code to the existing encounter_id vector
            pid_diagnosis_dict[pid][encounter_id].append(code)
        else:
            # Initialize a new encounter_id vector if encounter_id is not in the dictionary
            pid_diagnosis_dict[pid][encounter_id] = [code]
    else:
        # Initialize a new dictionary entry for pid and encounter_id
        pid_diagnosis_dict[pid] = {encounter_id: [code]}

# Initialize an empty list to store the final rows of the new DataFrame
new_rows = []

# Iterate through the dictionary to create rows for the new DataFrame
for pid, encounters in pid_diagnosis_dict.items():
    # Concatenate the vectors for each encounter_id
    concatenated_codes = []
    for encounter_id, codes in encounters.items():
        concatenated_codes += codes + ['SEP']

    # Create a new row with pid, encounter_id, and the aggregated code vector
    new_row = {'pid': pid, 'code': ['CLS'] + concatenated_codes}
    new_rows.append(new_row)

# Create the new DataFrame
patient_code_df = pd.DataFrame(new_rows)

# Print the new DataFrame
print(patient_code_df.head)

<bound method NDFrame.head of                          pid  \
0         IRB202001139_PAT_1   
1        IRB202001139_PAT_10   
2     IRB202001139_PAT_10001   
3     IRB202001139_PAT_10002   
4     IRB202001139_PAT_10009   
...                      ...   
9916   IRB202001139_PAT_9958   
9917   IRB202001139_PAT_9965   
9918   IRB202001139_PAT_9977   
9919   IRB202001139_PAT_9992   
9920   IRB202001139_PAT_9993   

                                                   code  
0     [CLS, 366.0, 366.0, 418.0, 705.0, 760.0, 838.0...  
1     [CLS, 1117.0, 1113.0, 65.0, 72.0, 366.0, 366.0...  
2     [CLS, 35.0, 920.0, 1696.0, SEP, 388.0, SEP, 43...  
3     [CLS, 492.0, 1531.0, 1534.0, SEP, 1707.0, SEP,...  
4     [CLS, 250.0, 579.0, 590.0, 590.0, 760.0, 824.0...  
...                                                 ...  
9916  [CLS, 159.0, 162.0, 164.0, 315.0, 479.0, 492.0...  
9917  [CLS, 119.0, 159.0, 162.0, 164.0, 172.0, SEP, ...  
9918  [CLS, 1.0, 1.0, 366.0, 392.0, 392.0, 498.0, 17...  
9919 

In [58]:
#print(patient_code_df.columns)
column_mapping = {
    'pid': 'deid_pat_ID',
    'code': 'diagnosis_code'
}

patient_code_df = patient_code_df.rename(columns=column_mapping)


In [59]:
final_merged_df = patient_age_df.merge(patient_code_df, on='deid_pat_ID')


In [60]:
print(final_merged_df.head(20))

               deid_pat_ID                                         age_vector  \
0       IRB202001139_PAT_1  [31, 31, 31, 31, 31, 31, 32, 33, 33, 33, 33, 3...   
1      IRB202001139_PAT_10  [69, 69, 70, 70, 70, 70, 70, 70, 71, 71, 71, 7...   
2   IRB202001139_PAT_10001  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
3   IRB202001139_PAT_10002  [19, 19, 20, 20, 21, 21, 21, 22, 22, 22, 22, 2...   
4   IRB202001139_PAT_10009  [66, 66, 66, 66, 67, 67, 67, 67, 67, 67, 67, 6...   
5   IRB202001139_PAT_10011                                               [80]   
6   IRB202001139_PAT_10014                                   [64, 65, 68, 68]   
7   IRB202001139_PAT_10018  [58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 5...   
8   IRB202001139_PAT_10020                   [83, 83, 83, 83, 83, 84, 84, 84]   
9   IRB202001139_PAT_10025  [48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 4...   
10  IRB202001139_PAT_10034  [48, 48, 49, 49, 49, 49, 49, 49, 49, 50, 50, 5...   
11  IRB202001139_PAT_10037  

In [61]:
final_merged_df.to_csv("/data/datasets/leyang.sun/merged_age_diagnosis.csv")

## Create Vocabulary for Age Codes (age2idx) and Diagnosis Codes (diagnosis2idx):

In [ ]:
import pandas as pd

# Example DataFrame
data = {'pid': [1, 1, 1, 2, 2],
        'encounter_id': [101, 101, 102, 201, 202],
        'diagnosis_code': ['1', '2', '3', '4', '5']}
df = pd.DataFrame(data)

# Initialize an empty dictionary to store aggregated vectors
pid_diagnosis_dict = {}

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    pid = row['pid']
    encounter_id = row['encounter_id']
    code = row['diagnosis_code']

    # Check if the pid is already in the dictionary
    if pid in pid_diagnosis_dict:
        # Check if encounter_id is already in the dictionary
        if encounter_id in pid_diagnosis_dict[pid]:
            # Append the diagnosis code to the existing encounter_id vector
            pid_diagnosis_dict[pid][encounter_id].append(code)
        else:
            # Initialize a new encounter_id vector if encounter_id is not in the dictionary
            pid_diagnosis_dict[pid][encounter_id] = [code]
    else:
        # Initialize a new dictionary entry for pid and encounter_id
        pid_diagnosis_dict[pid] = {encounter_id: [code]}

# Initialize an empty list to store the final rows of the new DataFrame
new_rows = []

# Iterate through the dictionary to create rows for the new DataFrame
for pid, encounters in pid_diagnosis_dict.items():
    # Concatenate the vectors for each encounter_id
    concatenated_codes = []
    for encounter_id, codes in encounters.items():
        concatenated_codes += codes + ['SEP']

    # Create a new row with pid, encounter_id, and the aggregated code vector
    new_row = {'pid': pid, 'code': ['CLS'] + concatenated_codes}
    new_rows.append(new_row)

# Create the new DataFrame
patient_code_df = pd.DataFrame(new_rows)

# Print the new DataFrame
print(patient_code_df)


In [ ]:
print(df)